# Reading the Train File for en-us text

Line to Article mapping has been stored for later optimizations on the training corpus.

In [121]:
# Returns {Line : Article}

def readnormalcorpus():
    linetest=0
    corpus={}
    fi=open("normal_test.txt","r")
    for line in fi:
        line=line.lower()
        linetest+=1
        if linetest > 1000:
            break
        corpus[[((line[:-1].split("\t")))[2]][0]]=[((line[:-1].split("\t")))[0]][0];
    return corpus

# Reading Train File for Simple Text

In [122]:
# Returns {Line : Article}

def readsimplecorpus():
    linetest=0
    corpus={}
    fi=open("simple_test.txt","r")
    for line in fi:
        line=line.lower()
        linetest+=1
        if linetest > 1000:
            break
        corpus[[((line[:-1].split("\t")))[2]][0]]=[((line[:-1].split("\t")))[0]][0];
    return corpus

# Building a Trigram Model

In [123]:
from itertools import tee, izip

def window(iterable, size):
    iters = tee(iterable, size)
    for i in xrange(1, size):
        for each in iters[i:]:
            next(each, None)
    return izip(*iters)

def tri_model():
    
    normal_corpus = readnormalcorpus()
    simple_corpus = readsimplecorpus()
    
    tgm_normal={}
    tgm_simple={}
        
    for i in normal_corpus.keys():
        for each in window(i.split(), 3):
            temp=list(each)
            if tgm_normal.has_key(tuple(temp)):
                tgm_normal[tuple(list(each))]+=1
            else:
                tgm_normal[tuple(list(each))]=1
    
                
    for i in simple_corpus.keys():
        for each in window(i.split(), 3):
            temp=list(each)
            if tgm_simple.has_key(tuple(temp)):
                tgm_simple[tuple(list(each))]+=1
            else:
                tgm_simple[tuple(list(each))]=1

    return tgm_normal,tgm_simple
    


# Cleaning the Normal Corpus from simple trigrams

In [124]:
def clean_normal_corpus(tgm_normal,tgm_simple):
    tgm_normal_cleaned={}
    for i in tgm_normal.keys():
        if i not in tgm_simple.keys():
            tgm_normal_cleaned[i]=tgm_normal[i]
    return tgm_normal_cleaned 

# First, Defining a Naive Bias Model for Classification

This function takes a trigram input and does a simple vs normal classification. As the trigram which is normal is also present in simple we classify it in simple.

In [125]:
def naive_classifier(sentence):
    tgm_normal,tgm_simple=tri_model()
    sentence=sentence.lower()
    score_simple=0
    score_normal=0
    total_simple=sum(tgm_simple.values())
    total_normal=sum(tgm_normal.values())
    print sentence
    for each in window(sentence.split(), 3):
        temp=list(each)
        if(tgm_simple.has_key(tuple(temp))):
            score_simple+=tgm_simple[tuple(temp)]*1.0/total_simple
        else :
            score_simple+=0.000003
        if(tgm_normal.has_key(tuple(temp))):
            score_normal+=tgm_normal[tuple(temp)]*1.0/total_normal
        else:
            score_normal+=0.000003
    
    print score_simple
    print score_normal
    if(score_normal>score_simple):
        print "NORMAL"
    else:
        print "SIMPLE"

sentence="Different kinds of the 1970s often have little or no concavity , whereas 1980s models have deeper concavities and steeper kicktails ."        
naive_classifier(sentence)
       
    

different kinds of the 1970s often have little or no concavity , whereas 1980s models have deeper concavities and steeper kicktails .
6e-05
9.71558045215e-05
NORMAL


# Defining a logestic Regression Model

In [126]:
import numpy as np
class LogisticRegression:
    def __init__(self, lr=0.01, num_iter=100000, fit_intercept=True, verbose=False):
        self.lr = lr
        self.num_iter = num_iter
        self.fit_intercept = fit_intercept
        self.verbose = verbose
    
    def __add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)
    
    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    def __loss(self, h, y):
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    
    def fit(self, X, y):
        if self.fit_intercept:
            X = self.__add_intercept(X)
        
        # weights initialization
        self.theta = np.zeros(X.shape[1])
        
        for i in range(self.num_iter):
            z = np.dot(X, self.theta)
            h = self.__sigmoid(z)
            gradient = np.dot(X.T, (h - y)) / y.size
            self.theta -= self.lr * gradient
            
            z = np.dot(X, self.theta)
            h = self.__sigmoid(z)
            loss = self.__loss(h, y)
                
            if(self.verbose ==True and i % 10000 == 0):
                print loss
    
    def predict_prob(self, X):
        if self.fit_intercept:
            X = self.__add_intercept(X)
    
        return self.__sigmoid(np.dot(X, self.theta))
    
    def predict(self, X):
        return self.predict_prob(X).round()

# Making X and Y for Logestic Regression

In [127]:
def makexandy(tgm_normal,tgm_simple,glo_idx):
    for i in tgm_normal:
        for j in i:
            if j not in glo_idx:
                glo_idx.append(j)
    for i in tgm_simple:
        for j in i:
            if j not in glo_idx:
                glo_idx.append(j)
    X=[]
    Y=[]
    
    for i in tgm_normal:
        temp=[]
        for j in i:
            temp.append(glo_idx.index(j))
        X.append(temp)
    
    for i in range(0,len(X)):
        Y.append(0)
    
    for i in tgm_simple:
        temp=[]
        for j in i:
            temp.append(glo_idx.index(j))
        X.append(temp)
        
    for i in range(0,len(tgm_simple)):
        Y.append(1)
        
    return X,Y
    

# Make X for test data and vectorize it (Using Trigrams)

In [128]:
from itertools import tee, izip

def window(iterable, size):
    iters = tee(iterable, size)
    for i in xrange(1, size):
        for each in iters[i:]:
            next(each, None)
    return izip(*iters)

def makeXfortest(sentence,glo_idx):
    X_temp=[]
    for each in window(sentence.split(), 3):
        X_temp.append(list(each))

    X=[]
    for i in X_temp:
        temp=[]
        for j in i:
            if j in glo_idx:
                temp.append(glo_idx.index(j))
            else:
                temp.append(-1)
        X.append(temp)
    return X

# Test 

In [142]:
import numpy as np
glo_idx=[]
tgm_normal,tgm_simple=tri_model()
#tgm_normal=clean_normal_corpus(tgm_normal,tgm_simple)
X,Y=makexandy(tgm_normal,tgm_simple,glo_idx)
X = np.array(X)
Y = np.array(Y)
model = LogisticRegression(lr=0.1, num_iter=3000)

model.fit(X, Y)

sentence="a city of Different kinds of formed by deck , truck the 1970s often have little or no concavity , whereas 1980s models have deeper concavities and steeper kicktails is a city."
X_Test=makeXfortest(sentence,glo_idx)

preds = model.predict(np.array(X))
correct=0
fp=0
fn=0
for i in range (0,len(preds)):
    if preds[i]==Y[i]:
        correct+=1
    elif Y[i]==0:
        fp+=1
    else:
        fn+=1
print "...........WITHOUT CLEANING..........."
print "_____ACCURACY_____"        
print correct*1.0/len(Y)*100

print "_____PRECISION_____"        
precision = correct*1.0/(correct+fp)*100
print precision

print "_____RECALL_____"        
recall = correct*1.0/(correct+fn)*100
print recall

print "_____F1 SCORE_____"
f1 = 1/((1/precision)+(1/recall))
print f1


########################### CLEAN

tgm_normal=clean_normal_corpus(tgm_normal,tgm_simple)
X,Y=makexandy(tgm_normal,tgm_simple,glo_idx)
X = np.array(X)
Y = np.array(Y)
model = LogisticRegression(lr=0.1, num_iter=3000)

model.fit(X, Y)

sentence="a city of Different kinds of formed by deck , truck the 1970s often have little or no concavity , whereas 1980s models have deeper concavities and steeper kicktails is a city."
X_Test=makeXfortest(sentence,glo_idx)

preds = model.predict(np.array(X))
correct=0
fp=0
fn=0
for i in range (0,len(preds)):
    if preds[i]==Y[i]:
        correct+=1
    elif Y[i]==0:
        fp+=1
    else:
        fn+=1
print ".............WITH CLEANING..........."
print "_____ACCURACY_____"        
print correct*1.0/len(Y)*100

print "_____PRECISION_____"        
precision = correct*1.0/(correct+fp)*100
print precision

print "_____RECALL_____"        
recall = correct*1.0/(correct+fn)*100
print recall

print "_____F1 SCORE_____"
f1 = 1/((1/precision)+(1/recall))
print f1

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in multiply
  app.launch_new_instance()
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in exp
  


...........WITHOUT CLEANING...........
_____ACCURACY_____
49.6829034321
_____PRECISION_____
59.7730259994
_____RECALL_____
74.6397117694
_____F1 SCORE_____
33.1921029676
.............WITH CLEANING...........
_____ACCURACY_____
40.0108769545
_____PRECISION_____
40.0108769545
_____RECALL_____
100.0
_____F1 SCORE_____
28.5769776069
